# Data Acquisition

## Web Scraping

In [18]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [19]:
def save_subgroup(dataframe, g_index, subgroup_name):
    """ save dataframe in csv """
    save_subgroup_filename = "".join([prefix, subgroup_name, ".csv.gz"])
    dataframe.to_csv(save_subgroup_filename, compression='gzip', encoding='UTF-8')

## players stats

scraping data form the basketball-reference website. 
The sites attempt a comprehensive approach to sports data. The data source relayalbe and cleanable.
I use the data of 2019-20 NBA Player Totals Stats to analysis.


>https://www.basketball-reference.com/leagues/NBA_2020_totals.html

In [36]:
def get_players_stats():
    year = 2020
#     url = "https://www.basketball-reference.com/leagues/NBA_2020_per_game.html"
    url = "https://www.basketball-reference.com/leagues/NBA_2020_totals.html"
    html = urlopen(url)
    soup = BeautifulSoup(html)
    #get headers
    soup.findAll('tr', limit=2)
    headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
    headers = headers[1:]
    #get rows
    rows = soup.findAll('tr')[1:]
    player_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]
    
    players_stats = pd.DataFrame(player_stats, columns = headers)
    return players_stats

In [37]:
players_stats = get_players_stats()

In [38]:
players_stats.shape

(649, 29)

In [39]:
players_stats.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Steven Adams,C,26,OKC,58,58,1564,262,443,.591,...,.590,196,347,543,141,50,65,86,111,633
1,Bam Adebayo,PF,22,MIA,65,65,2235,408,719,.567,...,.690,165,518,683,333,78,85,185,164,1053
2,LaMarcus Aldridge,C,34,SAS,53,53,1754,391,793,.493,...,.827,103,289,392,129,36,87,74,128,1001
3,Nickeil Alexander-Walker,SG,21,NOP,41,0,501,77,227,.339,...,.607,8,72,80,74,11,7,40,46,211
4,Grayson Allen,SG,24,MEM,30,0,498,79,176,.449,...,.857,5,61,66,43,6,1,23,36,221


In [42]:
players_stats.to_csv("stats.csv.gz", compression='gzip')

## players salary

ESPN is an American basic cable sports channel owned by ESPN Inc. The website recorde each season NBA Player Salaries.
I use the season of 2019-2020 salary.

> http://www.espn.com/nba/salaries

In [52]:
def get_totoal_pages_num():
    url = "http://www.espn.com/nba/salaries"
    html = urlopen(url)
    soup = BeautifulSoup(html)
    totoal_pages_num = soup.select('.page-numbers')[0].text[-2:]
    print("total pages num is : %s" % totoal_pages_num )
    return totoal_pages_num

In [53]:
def get_palyer_salary():
    pages = int(get_totoal_pages_num())
    df =None
    arr=[]
    for page_num in range(1,pages+1):
        per_page_url =  "http://www.espn.com/nba/salaries/_/page/{}".format(page_num)
        print(per_page_url)
        html = urlopen(per_page_url)
        soup = BeautifulSoup(html)
        soup.findAll('tr', limit=1)
        soup.findAll('tr', limit=1)[0].findAll('td')

        #get header
        headers = [th.getText() for th in soup.findAll('tr', limit=1)[0].findAll('td')]
        
        #get row
        rows = soup.findAll('tr')[1:]
        player_stats = [[td.getText() for td in rows[i].findAll('td')]
                for i in range(len(rows))]
    
        arr.extend(player_stats)
        df = pd.DataFrame(arr, columns = headers)
    return df

In [54]:
salary = get_palyer_salary()
salary.shape

total pages num is : 14
http://www.espn.com/nba/salaries/_/page/1
http://www.espn.com/nba/salaries/_/page/2
http://www.espn.com/nba/salaries/_/page/3
http://www.espn.com/nba/salaries/_/page/4
http://www.espn.com/nba/salaries/_/page/5
http://www.espn.com/nba/salaries/_/page/6
http://www.espn.com/nba/salaries/_/page/7
http://www.espn.com/nba/salaries/_/page/8
http://www.espn.com/nba/salaries/_/page/9
http://www.espn.com/nba/salaries/_/page/10
http://www.espn.com/nba/salaries/_/page/11
http://www.espn.com/nba/salaries/_/page/12
http://www.espn.com/nba/salaries/_/page/13
http://www.espn.com/nba/salaries/_/page/14


(564, 4)

In [55]:
salary.head()

,RK,NAME,TEAM,SALARY
0,1,"Stephen Curry, PG",Golden State Warriors,"$40,231,758"
1,2,"Chris Paul, PG",Oklahoma City Thunder,"$38,506,482"
2,3,"Russell Westbrook, PG",Houston Rockets,"$38,506,482"
3,4,"John Wall, PG",Washington Wizards,"$38,199,000"
4,5,"Kevin Durant, SF",Brooklyn Nets,"$38,199,000"


In [56]:
salary.to_csv("salary.csv.gz", compression='gzip')

In [57]:
salary['full-name'], salary['state'] = salary['NAME'].str.split(',', 1).str
salary.head()


,RK,NAME,TEAM,SALARY,full-name,state
0,1,"Stephen Curry, PG",Golden State Warriors,"$40,231,758",Stephen Curry,PG
1,2,"Chris Paul, PG",Oklahoma City Thunder,"$38,506,482",Chris Paul,PG
2,3,"Russell Westbrook, PG",Houston Rockets,"$38,506,482",Russell Westbrook,PG
3,4,"John Wall, PG",Washington Wizards,"$38,199,000",John Wall,PG
4,5,"Kevin Durant, SF",Brooklyn Nets,"$38,199,000",Kevin Durant,SF


In [58]:
# print(salary.dtypes)

In [59]:
def covert_currency(x): #1change salary formate
    if x == 0:
        return 0
    else:
        return x.replace("$","").replace(",","")
    
cust_str = salary['SALARY'].apply(lambda x:covert_currency(x)) #2

salary['SALARY'] = pd.to_numeric(cust_str, errors='coerce')

In [60]:
salary.head()

,RK,NAME,TEAM,SALARY,full-name,state
0,1,"Stephen Curry, PG",Golden State Warriors,40231758.0,Stephen Curry,PG
1,2,"Chris Paul, PG",Oklahoma City Thunder,38506482.0,Chris Paul,PG
2,3,"Russell Westbrook, PG",Houston Rockets,38506482.0,Russell Westbrook,PG
3,4,"John Wall, PG",Washington Wizards,38199000.0,John Wall,PG
4,5,"Kevin Durant, SF",Brooklyn Nets,38199000.0,Kevin Durant,SF
